### Use case example: Federated Analysis

As a clinical researcher, Alice wants to know if a specific treatment for a certain disease is effective.

Background: Alice has 12 patients with the diagnosis, of which 7 have received a specific treatment. We assume that the effect of the treatment is indicated by the lab value X. The mean values of X for both cohorts indicate that the treatment has an effect, but there is no statistical significance due to the small number of patients. 

Alice’s has searched for other sources of data and discovered that it is spread out over a network of edge nodes that support federated analysis through secure multiparty computation. Spread across all nodes there are a total of 1298 patients with the same diagnosis, of which 357 have received the specific treatment. By using more data, she can increase the statistical significance as the number of patients increases. The problem is that to protect the privacy of the people in the distributed database, Alice cannot simply download the data sets to her own computer and perform the analysis. Instead she must perform the calculations remotely at each edge node and then aggregate under homomoprhic encrytion, which is a cryptosystem that supports calculation on encrypted variables.

Challenge: The data that Alice needs is spread across multiple sources and there is no possibility of central collection due to privacy protection legislation. Even partial results from the analysis must be hidden from all parties, including Alice.

### Notation

Let's assume that there are $K$ edge nodes in the federated network. For the cohorts available at node $k$, we assume that $N_{1,k}$ patients have recieved the treatment, and that $N_{0,k}$ are untreated. For each patient $i$ at node $k$ we denote the measured lab value by $X_{i,k}$ and the treatment status by $Y_{i,k}$.

Alice wants to know the global average and variance for both cohorts so that she examine the hypothesis that the treatment is effective with statistical significans. That is, she wants to calculate, 

$$
\mu_0 = \mathrm{E}(X|Y=0),\quad \sigma_0^2 = \mathrm{Var}(X|Y=0) 
$$
$$
\mu_1 = \mathrm{E}(X|Y=1),\quad \sigma_1^2 = \mathrm{Var}(X|Y=1) 
$$

that is, the mean and variance for each cohort. She can estimate the above quantities for the patient data ditributed over the edge nodes by using secure multiparty computation based on homomorphic encryption.

Alice assumes the data is random, independent and normally distributed so she define sample mean and sample variance as
$$ 
\bar{X}_y = \mathrm{mean}(X|Y=y) = \frac{1}{n_y}\sum_i^{n_y}{x_{i,y}} 
$$
$$ 
\bar{S}^2_y = \mathrm{var}(X|Y=y)  = \frac{1}{n_y}\sum_i^{n_y}{( x_{i,y} - \bar{X}_y)^2 } 
$$

When calculating the variance each node needs to know the calculated mean.

Instead of calulating the mean and it sending back and do multiple roundtrips in the Sardin framework to calculate the variance, Alice expands the variance formula
 $$
 \frac{1}{n}\sum_i^n{( x_i - \bar{X})^2 } =
 $$
  $$ = \frac{1}{n}\sum_i^n{x_i}^2 - 2\cdot \bar{X} \cdot \frac{1}{n}\sum_i^n{x_i} + \bar{X}^2\frac{1}{n}\sum_i^n{1} = 
  $$
$$ = \frac{1}{n}\sum_i^n{x_i^2} - \bar{X}^2 
$$
For unbiased variance
$$ S^2  = \frac{1}{n-1}\sum_i^n{x_i^2} - \frac{n}{n-1}\bar{X}^2 
  $$

To calculate the mean and variance we need to aggregate 3 variables, the number of observations (patients), the variable, and the square of the variable 
$$\{n_y, \ x_{i,y}, \ x_{i,y}^2\}$$

### Prerequisites 

In [1]:
!pip install tenseal numpy scipy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.7 -m pip install --upgrade pip


In [2]:
from IPython.display import Latex, display
from scipy.stats import t as t_test
import tenseal as ts
import numpy as np
import math


In [13]:
# Create the context to encrypt data
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
context.generate_galois_keys()
context.global_scale = 2**40

encrypt_flag = True # True / False

### Create sudo data for toy example

In [14]:
from utils import getData

# Get data from patients without or without treatment
treatment = 0 # 0/1

# Number of edge nodes to participate
nodes = 7

# To manage the simulate data retrieval for each independent node, we keep all data in one vector 
node_data = []
for i in range(nodes):
    node_data.append(getData(treatment))

### Define functions to be run on edge nodes

In [15]:
# Return number of observation
def get_n(x):
    return len(x)

# Return sum of all observations
def get_xi(x):
    return x.sum()

# Return sum of squared observations
def get_xi_2(x):
    return x.dot(x)

# The function that will be doing the calculations on "each node"
def local_calculation(x):

    # The variables Alice are interested by in this example is the mean and variance
    # In the math section above we showed that in order for Alice to calculate mean and variance, this function
    #     will calculate the variables needed

    # INPUT: 
    #    x: observation data points from the local registry
    # OUTPUT:
    #    obs: number of observations
    #    xi: sum of observations
    #    xi_2: sum of squared observations

    n = get_n(x)
    xi = get_xi(x)
    xi_2 = get_xi_2(x)
    
    return n, xi, xi_2
    
# Simulating the distributed edge node calculations
encrypted_node_data = []
for data in node_data:
    
    # Run Alices functions on each edge node
    n, xi, xi_2 = local_calculation(data)

    if encrypt_flag:
        # Encrypt data the same each node
        calculated_data = ts.ckks_vector(context, [n, xi, xi_2])
    else:
        calculated_data = np.array([n, xi, xi_2])

    # Gather all encrypted data
    encrypted_node_data.append(calculated_data)

#####
<!-- låt säga vi har x data punkter att addera
$$
\frac{1}{2}+\frac{1}{4}+\frac{1}{8}+... = \sum^n_{k=1} (\frac{1}{2})^k = 1, n\rightarrow\inf
$$
n godtyckligt (5)
$$
\sum^5_{k=1} = \frac{1}{2}+\frac{1}{4}+\frac{1}{8}+\frac{1}{16}+\frac{1}{32}
$$
multiplicera med 32 (2^n)
$$
32\sum^5_{k=1} = 32*(\frac{1}{2}+\frac{1}{4}+\frac{1}{8}+\frac{1}{16}+\frac{1}{32}) = 16+8+4+2+1 = 31
$$ -->
<!-- 
Mängden additioner genom ett binary-tree med ett godtyckligt antal termer $2^n \,,\, n \in \mathbb{N}$ kan enkelt visas vara 
$$
2^n\sum^n_{k=1}(\frac{1}{2})^k = 2^n-1 \quad, \quad n<\infty
$$
vilket också är samma mängd additioner som för en naiv summation av samma antal.
Däremot skiljer det sig in hur CKKS adderar noise. Alice inser att i en naiv summering av en vector med CKKS ciphertexts
$$
\sum_i^N C_i = C_1 + C_2 + ...
$$
kommer öka noise linjärt. För att visa varför, skriver hon om föregående summa
$$
\sum_i^N C_i = \sum_i^{k-1} C_i + \sum_k^N C_k = C^{sum}_{k-1} + \sum_k^N C_k
$$
där $ C^{sum}_k $ är en partiell summa. Noise nivån i den partiella summan kan beskrivas som
$$
noise(C^{sum}_{k-1}) = {max_{noise}(C^{sum}_{k-2} \,,\, C_{k-1})} + \sigma
$$
där $\sigma$ är en godtycklig mängd noise
Alice tänker ett enklare fall där $\sigma$ är konstant och identiskt för alla additioner och hon kommer fram till att i detta fall kommer den slutliga noise nivån vara 
$$
noise(\sum_i^N C_i) = \sum_i^N ({max_{noise}(C^{sum}_{i-1} \,,\, C_i)} + \sigma ) = \sigma \cdot N
$$
Vid addition genom ett binary tree, för godtyckligt antal termar $x_0 \in \mathbb{N}^+$ och $Y := \{2^n | n \in \mathbb{N} \,,\, 2^n \leq x_0\}$ kan man dela upp additionerna till flera binary-trees där
$$
x_1 = \max(Y) 
$$
$$
x_2 = x_0-x_1
$$
är antalet termer att summeras i ett nästa binary-tree. Detta görs recursivt till man har kompletta binary-tree's.
Hur noise nivån ökar kan man se på nedansstående bild.
<img src="binary_tree.png" align="center"/>
Som Alice ser i bilden så är det antalet "lager" i trädet som avgör noise nivån för binary tree summation.
för ett tal $2^n \,,\, n \in \mathbb{N}$ så är antalet lager 
$$
log_2(2^n) = n
$$
Alice konstaterar att för ett generelt antal termer så är noise growth vid slutförd summation
$$
\lceil \, log_2(x) \, \rceil  \cdot \sigma
$$ -->

 The number of additions through a binary tree with an arbitrary number of terms $2^n \,,\, n \in \mathbb{N}$ can be easily shown to be
$$
2^n\sum^n_{k=1} \left(\frac{1}{2}\right)^k = 2^n - 1 \quad, \quad n<\infty
$$
which is the same number of additions as for a naive summation of the same quantity.

However, the difference lies in how CKKS manages the ciphertext modulus. Alice realizes that in a naive summation of a vector with CKKS ciphertexts
$$
\sum_i^N C_i = C_1 + C_2 + \ldots
$$
the circuit depth increases linearly, and consumes the computational budget. To demonstrate why, she rewrites the previous sum as
$$
\sum_i^N C_i = \sum_i^{k-1} C_i + \sum_k^N C_k = C^{sum}_{k-1} + \sum_k^N C_k
$$
where $C^{sum}_k$ is a partial sum. The modulus level in the partial sum can be described as
$$
\text{modulus}(C^{sum}_{k-1}) = \max_{\text{modulus}}(C^{sum}_{k-2} \,,\, C_{k-1}) + \Delta
$$
where $\Delta$ is the scaling factor.

Alice considers a simpler case where $\Delta$ is identical for all ciphertexts, and she concludes that in this scenario, the final modulus level will be
$$
\text{modulus}(\sum_i^N C_i) = \sum_i^N (\max_{\text{modulus}}(C^{sum}_{i-1} \,,\, C_i) + \Delta ) = \Delta \cdot N
$$

When adding through a binary tree, for an arbitrary number of terms $x_0 \in \mathbb{N}^+$ and $Y := \{2^n | n \in \mathbb{N} \,,\, 2^n \leq x_0\}$, the additions can be divided into several binary trees, where
$$
x_1 = \max(Y) 
$$
represents the number of terms in the first binary tree and
$$
x_2 = x_0 - x_1
$$

represent the number of terms to be summed in the next binary tree. This process is repeated recursively until complete binary trees are formed.

<img src="bilder/modulus.png" align="center"/>

The increase in modulus level can be observed in the provided image. Alice realises, the number of "layers" in the tree determines the circuit depth, or the modulus level for binary tree summation. For a number $2^n \,,\, n \in \mathbb{N}$, the number of layers is
$$
\log_2(2^n) = n
$$

Alice also notes that for a general number of terms, the modulus level growth after completing the summation is
$$
\lceil \, \log_2(x) \, \rceil \cdot \Delta
$$

This logic is also follows for multiplication in CKKS. In fact, for multiplication the modulus level growth faster and require much shorter circut depth

### Define the accumulator function

In [16]:
# Binary-tree summation
def binary_tree_accumulator(node_data):
    # [ .. [[c1 + c2] + [c3 + c4]] .. ]
    if not node_data:
        return 0
    elif len(node_data) == 1:
        return node_data[0]
    else:
        mid = len(node_data) // 2
        left_sum = binary_tree_accumulator(node_data[:mid])
        right_sum = binary_tree_accumulator(node_data[mid:])
        return left_sum + right_sum


# Naiv summation
def naiv_accumulator(node_data):
    accumulated_data = 0
    
    # [n_1, xi_1, xi^2_1]
    # + [n_2, xi_2, xi^2_2]
    # + ...
    # = [n, xi, xi^2]
    
    for data in node_data:
        accumulated_data += data
    return accumulated_data
    
encrypted_accumulated_data = binary_tree_accumulator(encrypted_node_data)
# encrypted_accumulated_data = naiv_accumulator(encrypted_node_data)

### Define the decode function

In [17]:
# The decode function will decrypt and reveal the accumulated data to Alice, only after all the previous steps have been completed succesfully
def decode(x):
    if encrypt_flag:
        return x.decrypt()
    return x

accumulated_data = decode(encrypted_accumulated_data)

### Define Alices function that will use the accumulated data

In [18]:
def calculate_mean_and_var(variables, unbiased): # vec = [n, x, x^2]
    
    n = round(variables[0])
    x = variables[1]
    x_2 = variables[2]

    mean = x/n
    
    # Calculate biased or unbiased variance based on the expanded formula
    if unbiased:
        var = x_2/(n-1) - (n/(n-1))*(mean**2)
    else:
        var = x_2/n - mean**2

    return n, mean, var

unbiased = True
n_0, mean_treatment_0, var_treatment_0 = calculate_mean_and_var(accumulated_data, unbiased=unbiased)
print(f"number observations = {n_0}, mean = {mean_treatment_0:.2f}, var = {var_treatment_0:.2f}, std = {var_treatment_0**(1/2):.2f}")

number observations = 938, mean = 85.34, var = 3908.13, std = 62.52


### Perform the same simulation for treated patients

In [19]:
treatment = 1 # 0/1

node_data = []
encrypted_node_data = []
for _ in range(nodes):
    
    # ---- Inside each node ---- #
    data = getData(treatment)
    n, xi, xi_2 = local_calculation(data)
    
    if encrypt_flag:
        data = ts.ckks_vector(context, [n, xi, xi_2])
    else:
        data = np.array([n, xi, xi_2])
    # ---- Inside each node ---- #

    # ---- Data gets sent to central Sardin server to be accumulated ---- #
    node_data.append(data)
accumulated_data = binary_tree_accumulator(node_data)

# ---- Decode data to Alice ---- # 
accumulated_data = decode(accumulated_data)

unbiased = True
n_1, mean_treatment_1, var_treatment_1 = calculate_mean_and_var(accumulated_data, unbiased=unbiased)
print(f"number observations = {n_1}, mean = {mean_treatment_1:.2f}, var = {var_treatment_1:.2f}, std = {var_treatment_1**(1/2):.2f}")

number observations = 357, mean = 75.30, var = 1743.65, std = 41.76


######
## Perform analysis with the retrieved data

Alice wants to investigate whether the two cohorts are significantly different or not and sets the null hypothesis as the two cohorts having the same mean
$$
H_0 : \mu_0 = \mu_1
$$

and the alternative hypoothesis as the two cohorts mean differs
$$
H_1 : \mu_0 \ne \mu_1
$$


Alice makes the assumptions that the data is random, independent and normally distributed but she also knows that the effect of the medication is not entierly clear so she can't make the assumtion of equal variance for the two cohorts

She then uses the Welche's t-test statistic
$$
t = \frac{\bar{X_0}-\bar{X_1}}{\sqrt{\frac{S^2_0}{n_0} + \frac{S^2_1}{n_1}}}
$$
where $S^2_0$ and $S^2_1$ is the unbiased sample variance, $ \bar{X_0} $, $\bar{X_1}$ is the sample mean and $n_0$, $n_1$ the number of data points

and the degrees of freedom is calculated as
$$
 d.f. = \frac{(\frac{S^2_0}{n_0} + \frac{S^2_1}{n_1})^2}{\frac{(S^2_0/n_0)^2}{n_0-1} + \frac{(S^2_1/n_1)^2}{n_1-1}}
$$

In [20]:
t = (mean_treatment_0 - mean_treatment_1) / (math.sqrt(var_treatment_0 / n_0 + var_treatment_1 / n_1))
df = ( var_treatment_0 / n_0 + var_treatment_1 / n_1 )**2 / ( ((var_treatment_0 / n_0)**2) / (n_0-1) + ((var_treatment_1 / n_1)**2) / (n_1-1) )

Latex(
"Alice calculates the t statistic" 
+ f"""\\begin{{equation*}}
t = \\frac{{ { "%.2f" % mean_treatment_0 } - { "%.2f" % mean_treatment_1} }}
{{ \\sqrt{{ \\frac{{ { "%.2f" % var_treatment_0} }}{{ {n_0} }} + \\frac{{ { "%.2f" % var_treatment_1} }}{{ { n_1 } }} }} }} = { "%.2f" % t }
\\end{{equation*}}
"""
+ "and the degrees of freedom" 
+  f"""\\begin{{equation*}}
d.f. = \\frac{{ ( \\frac{{{"%.2f" % var_treatment_0}}}{{{n_0}}} + \\frac{{{"%.2f" %var_treatment_1}}}{{{n_1}}})^2 }} 
{{ \\frac{{({"%.2f" % var_treatment_0}/{n_0})^2}}{{{n_0-1}}} + \\frac{{({"%.2f" % var_treatment_1}/{n_1})^2}}{{{n_1-1}}}}}
= { "%.2f" % df }
\\end{{equation*}}
"""
)

<IPython.core.display.Latex object>

In [21]:
p = 0.05

# For a two-tail test we use half of p to get the correct tabular value for the t statistic
t_score = t_test.isf(p/2, df)

# Can Alice reject the null hypothesis?
reject = True if t > t_score else False

# At what confidence level can Alice reject the null hypothesis
confidence_level = 1-t_test.sf(t, df)*2

display(Latex(
f"""Alice wants to use a {(1-p)*100}% confidence level for the test so she looks up the t-score in a table, $t_{{score}} = {"%.4f" % t_score } $ """
))
display(Latex(
f""" Since t {">" if reject else "≤"} $t_{{score}}$ Alice {"can" if reject else "can't"} reject the null hypothesis"""
))
display(Latex(
f""" Infact at a confidence level of {"%.2f" % (confidence_level*100)}% she can reject the null hypothesis """
))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>